Importing frameworks

In [156]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split  
from string import punctuation
import unidecode
from nltk.stem import RSLPStemmer
nltk.download('rslp')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\gustavo.selig\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Importing Database

In [157]:
resenha = pd.read_csv(r"C:\OneDrive-pessoal\OneDrive\Cursos\Alura -NLP\imdb-reviews-pt-br.csv")

# Selecting first 300 lines with pos
resenha_pos = resenha[resenha['sentiment']=="pos"]
resenha_pos = resenha_pos.reset_index(drop=True)
resenha_pos = resenha_pos.iloc[0:300, :]

# Selecting first 300 lines with neg
resenha_neg = resenha[resenha['sentiment']=="neg"]
resenha_neg = resenha_neg.reset_index(drop=True)
resenha_neg = resenha_neg.iloc[0:300, :]

# Concatening both dataframes
resenha = pd.concat([resenha_pos, resenha_neg])

Creating columns with dummies labels

In [158]:
# Creating column 'classificao'
classificacao = resenha["sentiment"].replace(["neg", "pos"], [0,1])
resenha["classificacao"] = classificacao

C:\Users\gustavo.selig\AppData\Local\Temp\ipykernel_25080\3391346603.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  classificacao = resenha["sentiment"].replace(["neg", "pos"], [0,1])


Creating bag of words

In [159]:
# Declarando o vetor
vetorizar = CountVectorizer(lowercase=False)

# Aplicando a coluna
bag_of_words = vetorizar.fit_transform(resenha['text_pt'])

# Transoforming to a pandas datraframe
df_bag = pd.DataFrame(
    bag_of_words.todense(),
    columns=vetorizar.get_feature_names_out()
)

display(df_bag.head(3))

,00,000,02,10,100,101,108,10Eliason,10Total,10s,...,últimas,último,últimos,única,únicas,único,únicos,úteis,útil,œAberrações
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Primeiro tratamento - removendo stopwords

In [160]:
# Declarando o tokenizer para separa palavras pelo espaço entre elas
token_espaco = tokenize.WhitespaceTokenizer()

# Lista de stopwords em portugues
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")

# Declarando a lista que sera usada para armazenar as frases tratadas e depois aplicá-la a coluna de tratamento do dataframe resenha
frase_processada = list()

# Realizando o tratamento
for opiniao in resenha.text_pt: # Iterando sobre cada linha do dataframe
    nova_frase = list() # Lista para armazenar palavras que não são stopwords
    palavras_texto = token_espaco.tokenize(opiniao) # Separando as palavras pelo espaço

    for palavra in palavras_texto: # Iterando sobre cada palavra da linha
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra) # Se a palavra nao for stopword é salva na lista nova_frase
    frase_processada.append(' '.join(nova_frase))

# Atribuindo a coluna do df resenha
resenha['tratamento_1'] = frase_processada

In [161]:
# Função para treinar um logistic regression e printar a acurácia
def classificar_texto(df, coluna_x, coluna_y):
    vetorizar = CountVectorizer(lowercase=False, max_features=50) # Declarando o vetor para o bag of words
    bag_of_words = vetorizar.fit_transform(df[coluna_x]) # Criando bag of words
    df_bag = pd.DataFrame( bag_of_words.todense(),columns=vetorizar.get_feature_names_out()) # Tranformando para pandas dataframe
    display(df_bag.head(3))
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                              df[coluna_y],
                                                              random_state = 42)
    regressao_logistica = LogisticRegression(solver = "lbfgs") # Declarando o algoritmo de ML
    regressao_logistica.fit(treino, classe_treino) # treinando o algoritmo
    return regressao_logistica.score(teste, classe_teste) # retornando o df de entrada acurácia


classificar_texto(resenha, "tratamento_1", "classificacao")

,Ele,Este,Eu,Mas,Não,Os,ainda,alguns,anos,apenas,...,tempo,ter,todos,tudo,tão,vai,ver,vez,vi,vida
0,0,2,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


0.7066666666666667

Segundo tratamento - removendo pontuacoes

In [162]:
# Lista com pontuacoes
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

# Declarando o tokenizer para separar palavras e acentos
token_pontuacao = tokenize.WordPunctTokenizer()

# Declarando a lista que sera usada para armazenar as frases tratadas e depois aplicá-la a coluna de tratamento do dataframe resenha
frase_processada = list()

# Realizando o tratamento
for opiniao in resenha['tratamento_1']: # Iterando sobre cada linha do dataframe
    nova_frase = list() # Lista para armazenar palavras que não são stopwords
    palavras_texto = token_pontuacao.tokenize(opiniao) # Separando as palavras e acentos

    for palavra in palavras_texto: # Iterando sobre cada palavra da linha
        if palavra not in pontuacao:
            nova_frase.append(palavra) # Se a palavra nao for acento, é salva na lista nova_frase
    frase_processada.append(' '.join(nova_frase))

# Atribuindo a coluna do df resenha
resenha['tratamento_2'] = frase_processada

classificar_texto(resenha, "tratamento_2", "classificacao")

,Ele,Este,Eu,Mas,Não,Os,ainda,alguns,anos,apenas,...,tempo,ter,todos,tudo,tão,vai,ver,vez,vi,vida
0,0,2,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


0.7066666666666667

Terceiro tratamento - removendo acentos

In [163]:
# Lista com os valores da coluna sem acento
sem_acentos = [unidecode.unidecode(texto) for texto in resenha['tratamento_2']]

# Atribuindo a coluna 'tratamento_3'
resenha['tratamento_3'] = sem_acentos

Quarto tratamento - removendo acentos do stopwords e removendo de novo elas e deixando tudo em minusculo

In [164]:
# Lista dos stopwords sem acento
stopwords_sem_acento = [unidecode.unidecode(texto) for texto in palavras_irrelevantes]

# Declarando a lista que sera usada para armazenar as frases tratadas e depois aplicá-la a coluna de tratamento do dataframe resenha
frase_processada = list()

# Realizando o tratamento
for opiniao in resenha['tratamento_3']: # Iterando sobre cada linha do dataframe
    nova_frase = list() 
    opiniao = opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao) 
    for palavra in palavras_texto: # Iterando sobre cada palavra da linha
        if palavra not in stopwords_sem_acento: 
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase)) # Se a palavra nao for stopword é salva na lista nova_frase

# Atribuindo a coluna do df resenha
resenha['tratamento_4'] = frase_processada

# Calculando acuracia
classificar_texto(resenha,"tratamento_3","classificacao")

,Ele,Este,Eu,Mas,Nao,Os,ainda,alguns,anos,apenas,...,sobre,tao,tempo,ter,todos,tudo,ver,vez,vi,vida
0,0,2,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,1,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


0.7066666666666667

Quinto tratamento - mantendo apenas o core das palavras via nltk stemmer

In [165]:
# Declarando o stemmer
stemmer = nltk.RSLPStemmer()

# Declarando a lista que sera usada para armazenar as frases tratadas e depois aplicá-la a coluna de tratamento do dataframe resenha
frase_processada = list()

# Realizando o tratamento
for opiniao in resenha['tratamento_4']: # Iterando sobre cada linha do dataframe
    nova_frase = list() 
    opiniao = opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao) 
    for palavra in palavras_texto: # Iterando sobre cada palavra da linha
        if palavra not in stopwords_sem_acento: 
            nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase)) # Se a palavra nao for stopword é salva na lista nova_frase

# Atribuindo a coluna do df resenha
resenha['tratamento_5'] = frase_processada

# Calculando acuracia
classificar_texto(resenha,"tratamento_5","classificacao")

,ach,aind,algum,ano,apen,assist,bem,bom,cas,cen,...,temp,tent,ter,the,tod,tud,ver,vez,vi,vid
0,0,0,1,0,0,0,1,1,0,0,...,0,1,0,0,1,0,0,0,2,0
1,0,1,3,0,0,0,1,2,1,0,...,0,0,0,0,0,0,0,2,0,0
2,0,0,1,0,0,0,2,0,1,1,...,0,1,1,0,0,0,0,0,0,0


0.7266666666666667

Sexto Tratamento - Aplicando tfidf para dar pesos relativos as palavras

In [166]:
# Declarando o tfidf
tfidf = TfidfVectorizer(lowercase = False, ngram_range=(1,2))

# Aplicando tfidf ao coluna coluna_x
tfidf_tratados = tfidf.fit_transform(resenha['tratamento_5'])

# Dividindo o dataset em treino e teste
treino, teste, classe_treino, classe_teste = train_test_split(tfidf_tratados, 
                                                              resenha['classificacao'],
                                                              random_state = 42)

# Declarando o algoritmo de ML
regressao_logistica = LogisticRegression(solver = "lbfgs")

# Treinando o algoritmo de ML
regressao_logistica.fit(treino,classe_treino)

# Calculando a acuracia via treino
acuracia_tfidf_tratados = regressao_logistica.score(teste,classe_teste)
acuracia_tfidf_tratados


0.8533333333333334